# **Kīauhōkū Model Offsets**
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zclaytor/kiauhoku/blob/master/notebooks/model_offsets.ipynb)
## This notebook is a tutorial on estimating the systematic uncertainties associated with various stellar model grids, as illustrated in Jamie Tayar et al. submitted 2020.
## The notebook is read-only. You can make changes, but they will not save unless you make your own copy. (So don't worry about messing it up for others!)

#### **Contact:**
- Jamie Tayar (jtayar@hawaii.edu) for information regarding the stellar model grids
- Zach Claytor (zclaytor@hawaii.edu) for information regarding kīauhōkū and detailed workings/upkeep of this notebook

#### **Resources:**
- kīauhōkū: https://github.com/zclaytor/kiauhoku
- models: https://zenodo.org/record/4307955

## First download and untar the model grids from Zenodo:

In [ ]:
!wget https://zenodo.org/record/4307955/files/eep_grids.tar.gz
!tar -xzvf eep_grids.tar.gz

## Next, install kiauhoku

In [ ]:
!pip install kiauhoku

In [ ]:
import numpy as np
import pandas as pd
import kiauhoku as kh

## Load grids, unify column names, and cast to interpolators

In [ ]:
# use grid points between ZAMS (201) and RGBump (605)
qstring = '0.6 <= initial_mass <= 2 and -1.0 <= initial_met <= 0.5 and 201 <= eep <= 605'

# Whether to fit evolved metallicity (True) or use the initial metallicity.
# False is probably fine if you're not on the giant branch.
evolve_met = False

# load grid, remove unwanted rows
yrec = kh.stargrid.from_parquet(path='eep_grids/yrec.pqt').query(qstring)
# set column names to some standard
yrec['mass'] = yrec['Mass(Msun)']
yrec['teff'] = 10**yrec['Log Teff(K)']
yrec['lum'] = 10**yrec['L/Lsun']
if evolve_met:
    yrec['met'] = np.log10(yrec['Zsurf']/yrec['Xsurf']/0.0253)
else:
    yrec['met'] = yrec.index.get_level_values('initial_met')
yrec['age'] = yrec['Age(Gyr)']
# set name for readability of output
yrec.set_name('yrec')
# cast to interpolator
yrec = yrec.to_interpolator()

mist = kh.stargrid.from_parquet(path='eep_grids/mist.pqt').query(qstring)
mist['mass'] = mist['star_mass']
mist['teff'] = 10**mist['log_Teff']
mist['lum'] = 10**mist['log_L']
if evolve_met:
    mist['met'] = mist['log_surf_z'] - np.log10(mist['surface_h1']*0.0173)
else:
    mist['met'] = mist.index.get_level_values('initial_met')
mist['logg'] = mist['log_g']
mist['age'] = mist['star_age'] / 1e9
mist.set_name('mist')
mist = mist.to_interpolator()

dart = kh.stargrid.from_parquet(path='eep_grids/dartmouth.pqt').query(qstring)
dart['mass'] = dart.index.to_frame()['initial_mass']
dart['teff'] = 10**dart['Log T']
dart['lum'] = 10**dart['Log L']
if evolve_met:
    dart['met'] = np.log10(dart['(Z/X)_surf']/0.0229)
else:
    dart['met'] = dart.index.get_level_values('initial_met')
dart['logg'] = dart['Log g']
dart['age'] = dart['Age (yrs)'] / 1e9
dart.set_name('dart')
dart = dart.to_interpolator()

gars = kh.stargrid.from_parquet(path='eep_grids/garstec.pqt').query(qstring)
gars['mass'] = gars['M/Msun']
gars['teff'] = gars['Teff']
gars['lum'] = 10**gars['Log L/Lsun']
if evolve_met:
    gars['met'] = np.log10(gars['Zsurf']/gars['Xsurf']/0.0245)
else:
    gars['met'] = gars.index.get_level_values('initial_met')
gars['age'] = gars['Age(Myr)'] / 1e3
gars.set_name('gars')
gars = gars.to_interpolator()

## Define fitting function
##### Iterate through list of grids to fit star

In [ ]:
def fit_all_grids(star, *args, **kwargs):
    gridnames = []
    models = []
    for gname, interp in zip(
        ['yrec', 'mist', 'dartmouth', 'garstec'],
        [yrec, mist, dart, gars]):
        model, fit = interp.gridsearch_fit(star, *args, **kwargs)
        if fit.success:
            gridnames.append(gname)
            models.append(
                model[['initial_mass', 'initial_met', 'eep', 'mass', 'teff', 'lum', 'met', 'logg', 'age']]
            )
    models = pd.concat(models, axis=1)
    models.columns = gridnames

    return models

def compute_statistics(models, exclude=None):
    stats = models.copy()
    if exclude is not None:
        stats = stats.drop(columns=exclude)

    mean = stats.mean(axis=1)
    stdev = stats.std(axis=1, ddof=1)
    max_offset = stats.max(axis=1) - stats.min(axis=1)

    stats['mean'] = mean
    stats['stdev'] = stdev
    stats['max offset'] = max_offset

    return stats

## Define stellar examples and run!

### $\pi$ Men

In [ ]:
piMen  = {'teff':6037, 'lum':1.444, 'met':0.08}
models = fit_all_grids(piMen, scale=(1000, 1, 0.1), tol=1e-6)
models

In [ ]:
stats = compute_statistics(models, exclude=None)
stats

### TOI 197

In [ ]:
toi197 = {'teff':5080, 'lum':5.15, 'met':-0.08}
models = fit_all_grids(toi197, scale=(1000, 1, 0.1), tol=1e-6)
models

In [ ]:
stats = compute_statistics(models, exclude=None)
stats

### Sun, using Teff and Luminosity

In [ ]:
sun1 = {'teff':5772, 'lum':1, 'met':0}
models = fit_all_grids(sun1, scale=(1000, 1, 0.1), tol=1e-6)
models

In [ ]:
stats = compute_statistics(models, exclude=None)
stats

### Sun, using Teff and logg

In [ ]:
sun2 = {'teff':5772, 'logg':4.44, 'met':0}
models = fit_all_grids(sun2, scale=(1000, 1, 0.1), tol=1e-6)
models

In [ ]:
stats = compute_statistics(models, exclude=None)
stats

### Sun, using Mass and Age

In [ ]:
sun3 = {'age': 4.57, 'mass':1, 'met':0}
models = fit_all_grids(sun3, scale=(1, 0.1, 0.1), tol=1e-6)
models

In [ ]:
stats = compute_statistics(models, exclude=None)
stats